In [12]:
import os
import logging
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader
from langchain.agents import initialize_agent, AgentType, Tool

from backend import config
from backend.feedback import FeedbackBot
from backend.resume_generation import ResumeGenerator

os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

1. QAchain input_document?
2. Agent
3. Job Description Length
4. Summary Length

In [2]:
feedback_bot = FeedbackBot()
resume_generator = ResumeGenerator()

In [11]:
llm = OpenAI(temperature=0)
tools = [
    Tool(
        name = "Update Prompts",
        func=feedback_bot.run,
        description="useful for when you need to re-generate the resume based on user request. Users may have special requirements and preferences and you will need to use this function to update the resume. For example, `more project experience` will be the input if you want to have more project experiences."
    ),
    Tool(
        name = "Generate Resume",
        func=resume_generator.run,
        description="useful for when you need to generate resume and the user does not have specific requirements. If the user have no new instructions, you will need to use this function to generate a new resume. For example, `` be will the input if you want to generate a resume."
    )
]

memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [3]:
job_description = """
About the job
Organization: Accenture Federal Services

Locations: Washington, DC

We are:  

Accenture Federal Services, helping our federal clients tackle their toughest challenges while unleashing their fullest potential…and then some. What makes our approach so unique? Operating from the nation’s capital, we bring together commercial innovation and leading-edge technologies to deliver an integrated and interactive experience that far exceeds expectations. How? Our passion meets purpose! Through our diverse culture and inclusive thinking, we embrace our employees' ideas taking them from concept to practical solutions. Not to mention, we sleep well at night knowing our work directly impacts and improves the way the world works. We keep our tech smarts sharp by providing abundant training and certification opportunities. Are you ready to learn and grow in a career, while making a difference? You are:  A lover of patterns. Perhaps you grew up putting together 1,000-piece puzzles, finding that tiny detail that gave away the perfect placement. You're on your A-game with the latest tools and methods, from statistics to machine learning. While others were struggling through stats, you were living your best life. You lead the charge to work with business stakeholders to understand their goals and determine how data can be used. What sets you apart? Unlike the rest, you see and apply your work through a human lens. You live to take any data mess from drab to fab and transform it into a story that the average Joe and Jane can understand.  The work:

Work with clients to create questions and define technology opportunities

Drive the exploration of data sources and analytic techniques to create new modeling approaches

Work with large data sets to solve complex problems

Apply statistical methods to develop analytic models

Potentially manage a technical team to solve clients’ solutions and expectations

Here's what experience you need:

Data programing languages to script, and analyze data using Python and/or R

Conducting complex queries in SQL or similar database language

Analytics and statistical modeling to collect, analyze and interpret data sets

Consulting or similar field requiring client collaboration, presentation, and delivery 

Extracting data, aggregating, and structuring data along different dimensions

Bachelor's degree

US Citizenship required; dual citizens not eligible

Compensation for roles at Accenture Federal Services varies depending on a wide array of factors including but not limited to the specific office location, role, skill set and level of experience. As required by local law, Accenture Federal Services provides a reasonable range of compensation for roles that may be hired in California, Colorado, New York City or Washington as set forth below and information on benefits offered is here.   

Role Location: Range of Starting Pay for role 

California: $90,700 - $198,800

Colorado: $90,700 - $171,700

New York City: $105,000 - $198,800

Washington: $96,600 - $182,800

Important information

An active security clearance or the ability to obtain one may be required for this role.

What We Believe

We have an unwavering commitment to diversity with the aim that every one of our people has a full sense of belonging within our organization. As a business imperative, every person at Accenture has the responsibility to create and sustain an inclusive environment.

Inclusion and diversity are fundamental to our culture and core values. Our rich diversity makes us more innovative and more creative, which helps us better serve our clients and our communities. Read more here

Equal Employment Opportunity Statement

Accenture is an Equal Opportunity Employer. We believe that no one should be discriminated against because of their differences, such as age, disability, ethnicity, gender, gender identity and expression, religion or sexual orientation.

All employment decisions shall be made without regard to age, race, creed, color, religion, sex, national origin, ancestry, disability status, veteran status, sexual orientation, gender identity or expression, genetic information, marital status, citizenship status or any other basis as protected by federal, state, or local law.

Accenture is committed to providing veteran employment opportunities to our service men and women.

For details, view a copy of the Accenture Equal Opportunity and Affirmative Action Policy Statement.

Requesting An Accommodation

Accenture is committed to providing equal employment opportunities for persons with disabilities or religious observances, including reasonable accommodation when needed. If you are hired by Accenture and require accommodation to perform the essential functions of your role, you will be asked to participate in our reasonable accommodation process. Accommodations made to facilitate the recruiting process are not a guarantee of future or continued accommodations once hired.

If you would like to be considered for employment opportunities with Accenture and have accommodation needs for a disability or religious observance, please call us toll free at 1 (877) 889-9009, send us an email or speak with your recruiter.

Other Employment Statements

Applicants for employment in the US must have work authorization that does not now or in the future require sponsorship of a visa for employment authorization in the United States.

Candidates who are currently employed by a client of Accenture or an affiliated Accenture business may not be eligible for consideration.

Job candidates will not be obligated to disclose sealed or expunged records of conviction or arrest as part of the hiring process.

The Company will not discharge or in any other manner discriminate against employees or applicants because they have inquired about, discussed, or disclosed their own pay or the pay of another employee or applicant. Additionally, employees who have access to the compensation information of other employees or applicants as a part of their essential job functions cannot disclose the pay of other employees or applicants to individuals who do not otherwise have access to compensation information, unless the disclosure is (a) in response to a formal complaint or charge, (b) in furtherance of an investigation, proceeding, hearing, or action, including an investigation conducted by the employer, or (c) consistent with the Company's legal duty to furnish information."""

In [4]:
template = """
You are given the detailed job description \n {job} \n.
Based on the job description, extract the required information, generate a concise summary.
Using the job description, extract the job/position title, company name, company position, specific teams/departments, any requirements (including the basic and preferred requirements), the description of expected responsibilities and expected personalities. 
IMPORTANT: You need to keep your words concise and not miss any important information. 

Past user requests:
None

generated resume:"""

In [7]:
rephrase_template = PromptTemplate(
    input_variables=[
        "job",
    ],
    template=template,
)
llm = OpenAI(temperature=0)

rephrase_chain = LLMChain(llm=llm,
                          prompt=rephrase_template,
                          verbose=False)

with get_openai_callback() as cb:
    res = rephrase_chain.run(job=job_description)
    print(cb)

Tokens Used: 1549
	Prompt Tokens: 1409
	Completion Tokens: 140
Successful Requests: 1
Total Cost (USD): $0.03098


In [5]:
verbose = False
background_path = './Samplebackground.docx'

In [6]:
with open(config.REPHRASE_PROMPT_TEMPLATE_PATH) as f:
    REPHRASE_PROMPT_TEMPLATE = f.read()
with open(config.JOB_SUMMARY_PROMPT_PATH) as f:
    JOB_SUMMARY_PROMPT = f.read()

with open(config.LATEX_PROMPT_TEMPLATE_1_PATH) as f:
    LATEX_PROMPT_TEMPLAT_1 = f.read()
with open(config.LATEX_PROMPT_TEMPLATE_2_PATH) as f:
    LATEX_PROMPT_TEMPLAT_2 = f.read()

with open(config.LATEX_HEADER_PATH) as f:
    latex_header = f.read()

with open(config.LATEX_PART1_PATH) as f:
    part1 = f.read()

with open(config.LATEX_PART2_PATH) as f:
    part2 = f.read()

llm = OpenAI(max_tokens=1200)

job_template = PromptTemplate(
    input_variables=[
        "job",
    ],
    template=JOB_SUMMARY_PROMPT,
)
job_summary_chain = LLMChain(llm=llm,
                                prompt=job_template,
                                verbose=verbose)
job_summary = job_summary_chain.run(job=job_description)
logging.info('The job summary is generated.')

loader = Docx2txtLoader(background_path)
background_info = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
background_info_doc = text_splitter.split_documents(background_info)

rephrase_template = PromptTemplate(
    input_variables=[
        "background",
        "job",
    ],
    template=REPHRASE_PROMPT_TEMPLATE,
)
rephrase_chain = LLMChain(llm=llm,
                            prompt=rephrase_template,
                            verbose=verbose)

results = []
for part in background_info_doc:
    res = rephrase_chain.run(background=part, job=job_summary)
    results.append(res)

background_info_generated = '\n\n'.join(results)
logging.info('The background summary is generated.')

In [18]:
background_info_generated

"\nName: Zara Patel \nAge: 27 \nEmail: zara.patel23@gmail.com \nPhone Number: +1 (555) 555-1212 \n\nBackground Summary:\nZara Patel is a highly motivated and experienced data scientist with a passion for problem-solving and personal growth. She earned a degree in Computer Science from UC Berkeley and has since worked in roles such as CTO and project manager in Silicon Valley. In her free time, she enjoys traveling, cooking, hiking, and playing the guitar, and also volunteers at a local animal shelter. \n\nWorking Experience:\nBanana Ink, Silicon Valley \nCTO, 2015-Current \n• Developed predictive maintenance model for industrial equipment, using Python and scikit-learn\n• Created a web application for technicians to view model predictions and schedule maintenance\n• Integrated model into industrial engineering team’s equipment maintenance process\n\nCustomer Segmentation for Retail Company, R \n• Leveraged SQL and other database languages to conduct complex queries \n• Applied analytic

In [26]:
with open( './backend/prompts/modified/background_section_1.txt') as f:
    BACKGROUND_SECTION_TEMPLATE_1 = f.read()
with open( './backend/prompts/modified/background_section_2.txt') as f:
    BACKGROUND_SECTION_TEMPLATE_2 = f.read()

In [27]:
results = []
background_template = PromptTemplate(
    input_variables=[
        "background",
    ],
    template=BACKGROUND_SECTION_TEMPLATE_1,
)

llm = OpenAI(max_tokens=1200, temperature=0.9)

background_section_chain = LLMChain(llm=llm,
                                prompt=background_template,
                                verbose=verbose)
background_section = background_section_chain.run(background=background_info_generated)

In [28]:
background_section

'\n\nName: Zara Patel \nAge: 27 \nEmail: zara.patel23@gmail.com \nPhone Number: +1 (555) 555-1212 \n\nEducation: \nComputer Science degree from UC Berkeley \n\nWorking Experience: \nBanana Ink, Silicon Valley \nCTO, 2015-Current \n• Developed predictive maintenance model for industrial equipment, using Python and scikit-learn \n• Created a web application for technicians to view model predictions and schedule maintenance \n• Integrated model into industrial engineering team’s equipment maintenance process \n\nCustomer Segmentation for Retail Company, R \n• Leveraged SQL and other database languages to conduct complex queries \n• Applied analytics and statistical modeling to collect, analyze, and interpret data sets \n• Developed new modeling approaches to explore data sources and technology opportunities \n• Managed a technical team to solve clients’ solutions and expectations \n\nSkills:\nPython, scikit-learn, web development, data visualization, machine learning, statistics, SQL, dat

In [29]:
results = []
background_template = PromptTemplate(
    input_variables=[
        "background",
    ],
    template=BACKGROUND_SECTION_TEMPLATE_2,
)

background_section_chain = LLMChain(llm=llm,
                                prompt=background_template,
                                verbose=verbose)
background_section = background_section_chain.run(background=background_info_generated)

In [30]:
background_section

"\nProject Experience:\n• Developed predictive maintenance model for industrial equipment, using Python and scikit-learn\n• Created a web application for technicians to view model predictions and schedule maintenance\n• Integrated model into industrial engineering team’s equipment maintenance process\n• Leveraged SQL and other database languages to conduct complex queries \n• Applied analytics and statistical modeling to collect, analyze, and interpret data sets \n• Developed new modeling approaches to explore data sources and technology opportunities\n• Managed a technical team to solve clients’ solutions and expectations\n• Conducted exploratory data analysis on customer purchasing patterns using R, resulting in improved customer segmentation\n• Applied unsupervised learning techniques such as k-means clustering and hierarchical clustering to segment customers based on their behavior\n• Developed a dashboard using Shiny to visualize the segments and provide insights to the business t